# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

#### --Import Statements--

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Datastore
from azureml.core.dataset import Dataset

from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails
from dataprocess import clean_data

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset consist information of patient that might be having heart disease. The target column has values 0 and 1 which determines if the patients has heart disease.
There are 14 attributes in the dataset as follows:
1. age
2. sex
3. chest pain type
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
14. target column, 0 = heart disease not present, 1 = heart disease present

After data is pulled, the data is processed for training in the following way:
- Standardization was performed on the column bloodpressure, thalac and cholestrol to save on computational cost.
- Rows with null values were dropped
- The age column was binned into 7 groups of '29-37', '38-45', '46-53', '54-61', '62-69' and '70-77'

After processing the data, the test input output and train input output data was set and fed to training.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

In [3]:
cpu_cluster_name = "cpu-cluster2"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


#### --Dataset Object--

In [42]:
key = "heart-dataset"
description_text = "Heart Disease dataset"

#-- Dataset Call
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00
mean,54.37,0.68,0.97,131.62,246.26,0.15,0.53,149.65,0.33,1.04,1.40,0.73,2.31,0.54
std,9.08,0.47,1.03,17.54,51.83,0.36,0.53,22.91,0.47,1.16,0.62,1.02,0.61,0.50
min,29.00,0.00,0.00,94.00,126.00,0.00,0.00,71.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,47.50,0.00,0.00,120.00,211.00,0.00,0.00,133.50,0.00,0.00,1.00,0.00,2.00,0.00
50%,55.00,1.00,1.00,130.00,240.00,0.00,1.00,153.00,0.00,0.80,1.00,0.00,2.00,1.00
75%,61.00,1.00,2.00,140.00,274.50,0.00,1.00,166.00,1.00,1.60,2.00,1.00,3.00,1.00
max,77.00,1.00,3.00,200.00,564.00,1.00,2.00,202.00,1.00,6.20,2.00,4.00,3.00,1.00


## --Preprocessing Data--

In [52]:
x_df, y_df = clean_data(df)

In [53]:
### Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(x_df, y_df, test_size=0.30)
df_train = pd.concat([X_train,Y_train], axis=1)
#print(df_train)
ds_train = Dataset.Tabular.register_pandas_dataframe(dataframe= df_train, target= Datastore.get_default(ws), name="train_data")
#dataset_1 = pd.to_Tabular(df_train) #Dataset.Tabular.from_pandas_dataframe(df_train)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/bd520736-c6c2-4d27-8427-af41fb843973/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The automl settings is configured as follows:
- Experiment timeout minutes determines how long experiment should run. In this case, its 20 minutes the experiment will run.
- Max concurrent iterations determines how many iterations should be running in parallel. We chose 5 iterations for this experiement.
- The primary metric to determine the performance of the runs being used here is AUC weighted.

In [54]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'AUC_weighted'
}

The automl config is configured as follows:
- The compute target is set to the one created
- The task is classification as the target column has values 0 and 1
- Early stopping is enabled
- The path and log path is assigned

In [55]:
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = 'classification',
                             training_data = ds_train,
                             label_column_name = "target",
                             path = './pipeline-project3',
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )

In [56]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_9878cecf-275e-4b33-9afc-454388c7fa66,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [57]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_9878cecf-275e-4b33-9afc-454388c7fa66,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

In [58]:
remote_run.get_output()

(Run(Experiment: automl-exp,
 Id: AutoML_9878cecf-275e-4b33-9afc-454388c7fa66_38,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                scale_pos_weight=1,
                                                                                                seed=None,
                              

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [59]:
automl_best_run = remote_run.get_best_child()

In [60]:
#TODO: Save the best model
amodel= automl_best_run.register_model(model_name='udacity-automl-project', model_path='./outputs/model.pkl')

In [61]:
temp_model = remote_run.get_output()
temp_model[-1]

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               scale_pos_weight=1,
                                                                                               seed=None,
                                                                                               silent=None,
                                                               

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [80]:
from azureml.core import Environment
import sklearn
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'azureml-automl-core',
    'azureml-automl-runtime',
    'packaging',
    'xgboost==0.90',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [85]:
with open('score.py') as f:
    print(f.read())

import joblib
import numpy as np
import os
import pickle
import json

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model
    print("HHHHH1")

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'model.pkl'
    print("HHHHH2")
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)
    print("Found model:",os.path.isfile(model_path))
    print("HHHHH3")
    print(model_path)
    print("what")
    #with open(model_path,'rb') as f:
     #   data = pickle.load(f)
    #print("HHHHH4")
    #with open('model.json','w+') as g

In [86]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Model

service_name = 'my-custom-env-service14'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = LocalWebservice.deploy_configuration(port=6789)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[amodel],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 5252a490f3754564adbddaca254ca3ff.azurecr.io
Logging into Docker registry 5252a490f3754564adbddaca254ca3ff.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 5252a490f3754564adbddaca254ca3ff.azurecr.io/azureml/azureml_7a680eced81598d47e771ec2cb704900
 ---> 08ac69bd1298
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 530f5af2034e
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY5NzFmNWFjLThhZjEtNDQ2ZS04MDM0LTA1YWNlYTI0NjgxZiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE0Mjg4MiIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE0Mjg4MiIsIndvcmtzcGFjZUlkIjoiNTI1MmE0OTAtZjM3NS00NTY0LWFkYmQtZGFjYTI1NGNhM2ZmIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in b0a4a0025a63
 ---> d3d986b27954
Step 4/5 : RUN mv '/var/azureml-app/tmpcpm3plvc.py' /var/azureml-app/main.py


In [87]:
print(service.get_logs())

2021-04-16T23:25:27,979674144+00:00 - rsyslog/run 
2021-04-16T23:25:27,980456448+00:00 - gunicorn/run 
2021-04-16T23:25:27,980770449+00:00 - iot-server/run 
2021-04-16T23:25:27,984466266+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
import json


input_payload = json.dumps({
    'data': X_test,
    'method': 'predict_prova'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)


NameError: name 'rest_endpoint' is not defined

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(output)

In [ ]:
service.delete()